# 🎮 Interactive Strategy Lab

Use the controls below to explore different strategy parameters and find the optimal setup for your portfolio.

### Strategies Included:
1. **Ladder Trend**: Buy on Blue Ladder Breakout, Sell on Breakdown.
2. **Parameter Tuning**: Adjust N1 (Short Term) and N2 (Long Term) EMAs.

---

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from core.indicators import TechnicalIndicators

# Global Data Cache to avoid re-downloading
DATA_CACHE = {}

def get_cached_data(ticker):
    if ticker not in DATA_CACHE:
        print(f"Downloading {ticker}...")
        # Fetch 5 years of data for backtesting
        df = yf.download(ticker, period="5y", interval="1d", progress=False)
        # Flatten MultiIndex if present
        if isinstance(df.columns, pd.MultiIndex):
            try:
                df.columns = df.columns.droplevel(1)
            except:
                pass
        DATA_CACHE[ticker] = df
    return DATA_CACHE[ticker].copy()

In [ ]:
# --- Strategy Logic ---

def run_backtest(ticker, n1, n2, initial_cash, take_profit_mult):
    df = get_cached_data(ticker)
    
    # Add Indicators
    df = TechnicalIndicators.add_ladder_indicator(df, n1=n1, n2=n2)
    # Note: Bottom Fishing Indicator can be added here if needed
    
    # Simulation Variables
    cash = initial_cash
    shares = 0
    equity_curve = []
    trades = []
    
    entry_price = 0
    
    for date, row in df.iterrows():
        price = row['Close']
        
        # Current Value
        current_val = cash + (shares * price)
        
        # --- Signals ---
        
        # Buy Condition: Price > Blue Ladder Top (Breakout)
        breakout = price > row['ladder_blue_top']
        
        # Sell Condition 1: Ladder Breakdown (Stop Loss)
        breakdown = price < row['ladder_blue_bottom']
        
        # Sell Condition 2: Take Profit (Dynamic)
        hit_tp = (shares > 0) and (price >= entry_price * take_profit_mult)
        
        # EXECUTION
        if shares == 0 and breakout and cash > 0:
            # BUY
            buy_shares = cash / price
            shares = buy_shares
            cash = 0
            entry_price = price
            trades.append({'date': date, 'action': 'BUY', 'price': price, 'reason': 'Breakout'})
            
        elif shares > 0:
            if hit_tp:
                # SELL (TP)
                proceeds = shares * price
                cash = proceeds
                shares = 0
                trades.append({'date': date, 'action': 'SELL', 'price': price, 'reason': 'Take Profit'})
            elif breakdown:
                # SELL (Stop)
                proceeds = shares * price
                cash = proceeds
                shares = 0
                trades.append({'date': date, 'action': 'SELL', 'price': price, 'reason': 'Ladder Stop'})
                
        equity_curve.append(current_val)
        
    df['Equity'] = equity_curve
    return df, trades

In [ ]:
# --- UI Controls ---

style = {'description_width': 'initial'}

w_ticker = widgets.Text(value='TQQQ', description='Ticker:', style=style)
w_n1 = widgets.IntSlider(value=26, min=5, max=50, description='Blue Ladder (N1):', style=style)
w_n2 = widgets.IntSlider(value=89, min=50, max=200, description='Yellow Ladder (N2):', style=style)
w_tp = widgets.FloatSlider(value=3.0, min=1.1, max=10.0, step=0.1, description='Take Profit (x):', style=style)
w_run = widgets.Button(description='Run Backtest', button_style='success', icon='play')

out = widgets.Output()

def on_run_click(b):
    from IPython.display import clear_output  # Local import to fix NameError
    with out:
        clear_output(wait=True)
        print(f"Running analysis for {w_ticker.value}... Please wait.")
        
        try:
            df_res, trades = run_backtest(
                ticker=w_ticker.value,
                n1=w_n1.value,
                n2=w_n2.value,
                initial_cash=10000,
                take_profit_mult=w_tp.value
            )
            
            if df_res.empty:
                print("Error: No data found.")
                return
            
            # --- Metrics ---
            init_eq = df_res['Equity'].iloc[0]
            final_eq = df_res['Equity'].iloc[-1]
            ret = (final_eq - init_eq) / init_eq
            
            # Calculate Drawdown
            roll_max = df_res['Equity'].cummax()
            drawdown = (df_res['Equity'] - roll_max) / roll_max
            max_dd = drawdown.min()
            
            print(f"\n=== 📊 Results for {w_ticker.value} ===")
            print(f"Total Return: {ret:.2%}")
            print(f"Max Drawdown: {max_dd:.2%}")
            print(f"Total Trades: {len(trades)}")
            print(f"Final Equity: ${final_eq:,.2f}")
            
            # --- Plots ---
            plt.style.use('ggplot')
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True, height_ratios=[3, 1])
            
            # Plot 1: Price & Ladder
            ax1.plot(df_res.index, df_res['Close'], label='Price', color='black', alpha=0.7, linewidth=1)
            
            # Plot Ladders
            ax1.plot(df_res.index, df_res['ladder_blue_top'], 'b--', alpha=0.4, linewidth=1, label=f'Blue Top ({w_n1.value})')
            ax1.plot(df_res.index, df_res['ladder_blue_bottom'], 'b--', alpha=0.4, linewidth=1)
            ax1.fill_between(df_res.index, df_res['ladder_blue_top'], df_res['ladder_blue_bottom'], color='blue', alpha=0.05)
            
            ax1.plot(df_res.index, df_res['ladder_yellow_top'], 'orange', linestyle=':', alpha=0.6, linewidth=1.5, label=f'Yellow Top ({w_n2.value})')

            # Plot Trade Markers
            for t in trades:
                if t['action'] == 'BUY':
                    ax1.scatter(t['date'], t['price'], marker='^', color='green', s=100, zorder=5)
                elif t['action'] == 'SELL':
                    color = 'red' if t['reason'] == 'Ladder Stop' else 'purple'
                    marker = 'v' if t['reason'] == 'Ladder Stop' else 'x'
                    ax1.scatter(t['date'], t['price'], marker=marker, color=color, s=100, zorder=5)

            ax1.set_title(f"{w_ticker.value} - Strategy Analysis")
            ax1.legend(loc='upper left')
            ax1.grid(True)
            
            # Plot 2: Equity Curve
            ax2.plot(df_res.index, df_res['Equity'], color='darkblue', linewidth=2, label='Strategy Equity')
            ax2.set_ylabel('Portfolio Value ($)')
            ax2.set_yscale('log')
            ax2.legend()
            ax2.grid(True)
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Error during execution: {str(e)}")
            import traceback
            traceback.print_exc()

w_run.on_click(on_run_click)

# Layout
controls = widgets.VBox([
    widgets.HBox([w_ticker, w_run]),
    widgets.HTML("<br><b>Strategy Parameters:</b>"),
    widgets.HBox([w_n1, w_n2]),
    widgets.HBox([w_tp])
])

display(controls)
display(out)